In [2]:
import pandas as pd
import numpy as np
import gurobipy as gp
from gurobipy import GRB

In [3]:
df = pd.read_csv('data_20180119_20180420.csv',index_col=0)
df_0102 = df[df['t0'] == '2018-01-02']
C1 = df_0102.set_index('K1')['C1']
C2 = df_0102.set_index('K1')['C2']

### Grid for stock price
current setting S_{T2}, S_{T1} from range [1100,1300] and 20\*20 discretization

In [11]:
S_min = 1100
S_max = 1300
N_j = 20
S = np.linspace(S_min, S_max, num=21)

### Optimization Variables
variables to optimize: lambda_1, lambda_2, d, \Delta_1, \Delta_0

In [51]:
m = gp.Model("Option Pricing")
lamb1 = m.addVars(df_0102['K1'],name="lamb1")
lamb2 = m.addVars(df_0102['K1'],name="lamb2")
d = m.addVars(['d'],name="d")
delta0 = m.addVars(['delta0'],name="delta0")
delta1 = m.addVars(S,name="delta1")

### Model Construction
objective: min$d+\sum_{i=1}^{N_1}\lambda_{i,1}\Pi_{i,1}+\sum_{i=1}^{N_2}\lambda_{i,2}\Pi_{i,2}$

constrains: for each grid point (S_i,S_j), we have

$d+\sum_{n=1}^{N_1}\lambda_{n,1}(S_i-K_{n,1})^++\sum_{n=1}^{N_2}\lambda_{n,2}(S_j-K_{n,2})^++\Delta_{0}\cdot(S_i-S_0)+\Delta_{1}^i\cdot(S_j-S_i) >= \Phi(S_i,S_j)$

Current payoff $\Phi(S_i,S_j) = S_j - S_i$

In [52]:
m.setObjective(d['d']+sum(lamb1[r]*C1[r]+lamb2[r]*C2[r] for r in df_0102['K1']))
m.ModelSense = GRB.MINIMIZE

In [53]:
def payoff(S1,S2):
    return S2-S1
S_0 = df_0102['Adj_S0'].unique()[0]
for S_i in S:
    for S_j in S:
        m.addConstr(d['d']+sum(lamb1[r]*max(S_i-r,0)+lamb2[r]*max(S_j-r,0) for r in df_0102['K1'])+delta0['delta0']*(S_i-S_0)+delta1[S_i]*(S_j-S_i) >= payoff(S_i,S_j),'C_{}{}'.format(S_i,S_j))
        m.update()

In [54]:
m.optimize()

Gurobi Optimizer version 11.0.1 build v11.0.1rc0 (win64 - Windows 11.0 (22000.2))

CPU model: AMD Ryzen 7 5800H with Radeon Graphics, instruction set [SSE2|AVX|AVX2]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 441 rows, 63 columns and 9912 nonzeros
Model fingerprint: 0x697c7187
Coefficient statistics:
  Matrix range     [2e-13, 3e+02]
  Objective range  [8e-01, 4e+02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+01, 2e+02]
         Consider reformulating model or setting NumericFocus parameter
         to avoid numerical issues.
Presolve removed 52 rows and 5 columns
Presolve time: 0.01s
Presolved: 389 rows, 58 columns, 8596 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   1.011250e+03   0.000000e+00      0s
      19    0.0000000e+00   0.000000e+00   0.000000e+00      0s

Solved in 19 iterations and 0.02 seconds (0.01 work units)
Optimal objective  0.000000000e+00
